In [1]:
using JuMP, Gurobi, DataFrames

In [2]:
df = readtable("communities-and-crime.csv",header=false)
X = Matrix(df[1:end - 1])
y = df[end];

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] #readtable#232(::Bool, ::Char, ::Array{Char,1}, ::Char, ::Array{String,1}, ::Array{String,1}, ::Array{String,1}, ::Bool, ::Int64, ::Array{Symbol,1}, ::Array{Any,1}, ::Bool, ::Char, ::Bool, ::Int64, ::Array{Int64,1}, ::Bool, ::Symbol, ::Bool, ::Bool, ::DataFrames.#readtable, ::String) at C:\Users\utilisateur\.julia\v0.6\DataFrames\src\deprecated.jl:1045
 [3] (::DataFrames.#kw##readtable)(::Array{Any,1}, ::DataFrames.#readtable, ::String) at .\<missing>:0
 [4] include_string(::String, ::String) at .\loading.jl:522
 [5] include_string(::Module, ::String, ::String) at C:\Users\utilisateur\.julia\v0.6\Compat\src\Compat.jl:88
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\utilisateur\.julia\v0.6\IJulia\src\execute_request.jl:180
 [7] (::Compat.#inner#14{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at C:\Users\utilisateur\.julia\v0.6\Compat\src\Compat.jl:332
 [8] eventloop(::ZMQ.Sock

In [3]:
# Split into training, validation and test sets (50%/25%/25%)
n = length(y)
val_start = round(Int, 0.50 * n)
test_start = round(Int, 0.75 * n)
train_X = X[1:val_start - 1, :]
train_y = y[1:val_start - 1]
val_X = X[val_start:test_start - 1, :]
val_y = y[val_start:test_start - 1]
test_X = X[test_start:end, :]
test_y = y[test_start:end];

In [4]:
#See the size of training set and test set
println("Size of training set:",size(train_X),size(train_y))
println("Size of validation set:",size(val_X),size(val_y))
println("Size of test set:",size(test_X),size(test_y))

Size of training set:(60, 122)(60,)
Size of validation set:(31, 122)(31,)
Size of test set:(31, 122)(31,)


In [5]:
# write the training functions for different types of linear regressions

##### standard linear regression #####
function standardlinear(X, y)
    # OutputFlag=0 to hide output from solver
    m = Model(solver=GurobiSolver(OutputFlag=0))    
    p = size(X, 2) #nb of columns

    #variables    
    @variable(m, t)
    @variable(m, β[1:p])
    
    # Constraints
    @constraint(m, norm(y - X * β) <= t)
    
    # Objective
    @objective(m, Min, t)

    solve(m)
    return getvalue(β)
end

##### lasso linear regression #####
function lassolinear(X, y, ρ)
    # OutputFlag=0 to hide output from solver
    m = Model(solver=GurobiSolver(OutputFlag=0))
    
    p = size(X, 2) #nb of columns

   
    
    #variables    
    @variable(m, t)
    @variable(m, β[1:p])
    @variable(m, a[1:p])
    
    # Constraints
    @constraint(m, norm(y - X * β) <= t)
    @constraint(m, -a[1:p] .<= β[1:p])
    @constraint(m, β[1:p] .<= a[1:p])
    @constraint(m, a[1:p] .>= 0)
    

    # Objective
    @objective(m, Min, t + ρ * sum(a[j] for j = 1:p))

    solve(m)

    return getvalue(β)
end

function lassolinear(X, y, ρ)
    # OutputFlag=0 to hide output from solver
    m = Model(solver=GurobiSolver(OutputFlag=0))
    
    p = size(X, 2) #nb of columns

   
    
    #variables    
    @variable(m, t)
    @variable(m, β[1:p])
    @variable(m, a[1:p])
    
    # Constraints
    @constraint(m, norm(y - X * β) <= t)
    @constraint(m, -a[1:p] .<= β[1:p])
    @constraint(m, β[1:p] .<= a[1:p])
    @constraint(m, a[1:p] .>= 0)
    

    # Objective
    @objective(m, Min, t + ρ * sum(a[j] for j = 1:p))

    solve(m)

    return getvalue(β)
end

##### ridge linear regression #####
function ridgelinear(X, y, ρ)
    # OutputFlag=0 to hide output from solver
    m = Model(solver=GurobiSolver(OutputFlag=0))
    p = size(X, 2) #nb of columns

    # Variables
    @variable(m, t)
    @variable(m, u)
    @variable(m, β[1:p])
    
    # Constraints
    @constraint(m, norm(y - X * β) <= t)
    @constraint(m, norm(β) <= u)

    # Objective
    @objective(m, Min, t + ρ * u)

    solve(m)

    return getvalue(β)
end

ridgelinear (generic function with 1 method)

In [6]:
function findBestRho(train_X,
                    train_y,
                    val_X,
                    val_y, 
                    rho_list)
    p = size(train_X, 2)
    k = length(rho_list)
    #instantiate arrays
    β_lasso_list = zeros(k, p)
    β_ridge_list = zeros(k, p)
    lasso_scores = zeros(k)
    ridge_scores = zeros(k)
    
    for i in 1:length(rho_list)
        # training on train sets for both regression methods
        
        β_lasso_list[i, :] = lassolinear(train_X, train_y, rho_list[i])
        #println("\nβ_lasso for rho =", rho_list[i], β_lasso_list[i, :])
        β_ridge_list[i, :] = ridgelinear(train_X, train_y, rho_list[i])
        #println("\nβ_ridge for rho =", rho_list[i], β_ridge_list[i, :])
        
        # performance metrics on validation sets for both regression methods
        lasso_scores[i] = norm(val_y - val_X * β_lasso_list[i, :])
        ridge_scores[i] = norm(val_y - val_X * β_ridge_list[i, :])
        
        
    end
    #println(lasso_scores)
    #println(ridge_scores)
    argmin_lasso = indmin(lasso_scores)
    argmin_ridge = indmin(ridge_scores)
    
    return rho_list[argmin_lasso], rho_list[argmin_ridge]
end

findBestRho (generic function with 1 method)

In [7]:
rho_list = [0.001, 0.01, 0.1, 1, 2]
best_rho = findBestRho(train_X, train_y, val_X, val_y, rho_list)
println("Best rho for lasso: ", best_rho[1] )
println("Best rho for ridge: ", best_rho[2])

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Best rho for lasso: 1.0
Best rho for ridge: 2.0


In [8]:
# retrain the whole model with training and validation sets together

new_train_X = vcat(train_X, val_X)
new_train_y = vcat(train_y, val_y)

# find best beta for standard linear regression
β_standard = standardlinear(new_train_X, new_train_y)
println("\nBest β for standard linear regression is: ", β_standard)

# find best beta for lasso

ρ_lasso = best_rho[1]
β_lasso = lassolinear(new_train_X, new_train_y, ρ_lasso)
println("\nBest β for lasso is: ", β_lasso)

# find best beta for ridge

ρ_ridge = best_rho[2]
β_ridge = ridgelinear(new_train_X, new_train_y, ρ_ridge)
println("\nBest β for ridge is: ", β_ridge)

Academic license - for non-commercial use only

Best β for standard linear regression is: [-0.0527188, 0.57872, -0.649981, -0.564125, -0.120323, -0.201748, 1.02188, 0.0263513, -0.636356, -0.885288, -0.111684, 0.393866, 0.797354, -0.0167027, 0.371823, 0.994967, -0.280043, 0.234241, 0.618182, -0.329469, 1.20287, 0.109965, -0.477511, 0.0487904, 0.0559053, 0.660927, 0.0281862, 0.51628, -0.792707, -1.08652, 1.20646, 0.661044, 0.31798, 0.00806002, -0.38317, -0.0977316, 0.197213, 0.16222, 0.151204, 0.222365, 0.386112, 0.759026, -0.23994, -1.19354, -0.71622, 0.0450472, 0.766182, 0.865574, -1.18412, -0.464879, -0.853547, -0.232746, -0.251155, 0.0510877, -0.0455335, 0.577966, 0.0274876, -1.20301, -0.403138, 0.998665, 0.738847, 1.37342, 0.421307, 0.130119, -0.540267, 1.15078, -0.233275, -0.446227, -0.502373, 0.855415, -0.140365, -0.683184, -0.507078, 0.186683, 0.129039, -0.525952, -0.792916, 0.280602, -0.033614, -1.34664, -0.0466065, 0.606263, -1.24193, -0.279984, 0.81491, 0.103892, -0.510585, -0

In [9]:
# score standard linear regression
score_standard = norm(test_y - test_X * β_standard)
println("Standard linear regression score: ", score_standard)

# score lasso
score_lasso = norm(test_y - test_X * β_lasso)
println("Lasso score: ", score_lasso)

# score ridge
score_ridge = norm(test_y - test_X * β_ridge)
println("Ridge score: ", score_ridge)

# baseline
train_y_mean = mean(new_train_y) #use mean on training and validation sets
score_baseline = norm(test_y - train_y_mean)
println("Baseline score: ", score_baseline)

# compare scores of regression with the baseline model
println("\nRelative gap standard linear regression % baseline: ", (score_baseline - score_standard)*100/score_baseline, " %" )
println("Relative gap lasso % baseline: ", (score_baseline - score_lasso)*100/score_baseline, " %" )
println("Relative gap ridge % baseline: ", (score_baseline - score_ridge)*100/score_baseline, " %" )

Standard linear regression score: 2.5479228082978107
Lasso score: 0.7833980234161126
Ridge score: 0.750864457615523
Baseline score: 1.4589372704759014

Relative gap standard linear regression % baseline: -74.64238249713678 %
Relative gap lasso % baseline: 46.30351562952598 %
Relative gap ridge % baseline: 48.533465227700084 %
